In [35]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [29]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("swaroopkml/cifar10-pngs-in-folders")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\anton\.cache\kagglehub\datasets\swaroopkml\cifar10-pngs-in-folders\versions\1


In [30]:
test_path = path + '/cifar10/cifar10/test'
train_path = path + '/cifar10/cifar10/train'

In [31]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [32]:
test_dataset = datasets.ImageFolder(
    root=test_path,
    transform=test_transforms,
)
train_dataset = datasets.ImageFolder(
    root=train_path,
    transform=train_transforms,
)

In [33]:
class MyCNN(torch.nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding='same')
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding='same')
        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, padding='same')
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = torch.nn.Linear(128 * 4 * 4, 256)
        self.fc2 = torch.nn.Linear(256, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))#3x32x32 -> 32x32x32
        x = self.pool(x)#32x32x32 -> 32x16x16
        x = torch.relu(self.conv2(x))#32x16x16 -> 64x16x16
        x = self.pool(x)#64x16x16 -> 64x8x8
        x = torch.relu(self.conv3(x))#64x8x8 -> 128x8x8
        x = self.pool(x)#128x8x8 -> 128x4x4
        x = x.flatten(start_dim=1)
        x = torch.relu(self.fc1(x))#128*4*4 -> 256
        x = torch.softmax(self.fc2(x), 1)#256 -> 10
        return x

In [34]:
#fix seeds
torch.manual_seed(42)

In [36]:
writer = SummaryWriter("logs")

In [37]:
def train(num_epochs, model, train_dataloader, criterion, optimizer, device):
    model.train()
    report = []
    for num_epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            writer.add_scalar('Batch Loss', loss.item())
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_dataloader)
        writer.add_scalar('Training Loss', epoch_loss)
        print(f'Epoch {num_epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        report.append(epoch_loss)
    print('Training complete')
    return report

In [38]:
train(
    num_epochs=5,
    model=MyCNN(),
    train_dataloader=DataLoader(train_dataset, batch_size=64, shuffle=True),
    criterion=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(MyCNN().parameters(), lr=0.001),
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

  0%|          | 0/782 [00:00<?, ?it/s]

100%|██████████| 782/782 [02:43<00:00,  4.78it/s]


Epoch 1/5, Loss: 147.2244


100%|██████████| 782/782 [02:37<00:00,  4.98it/s]


Epoch 2/5, Loss: 147.2246


...

100%|██████████| 782/782 [02:35<00:00,  5.04it/s]


Epoch 4/5, Loss: 147.2247


100%|██████████| 782/782 [02:34<00:00,  5.05it/s]

Epoch 5/5, Loss: 147.2246
Training complete


[147.22441530837426,
 147.2245747793056,
 147.2246874006813,
 147.2246704004,
 147.22460847254604]